In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from re import split

In [2]:
def intersect(df1,df2):
    loc1 = (df1.columns).isin(df2.columns)
    loc1 = np.where(loc1 == True)[0]
    df1 = df1.iloc[:,loc1]


    loc2 = (df2.columns).isin(df1.columns)
    loc2 = np.where(loc2 == True)[0]
    df2 = df2.iloc[:,loc2]
    return df1, df2

def col_addname(df,name):
    col = df.columns
    col = col.map(lambda x: name + x)
    df.columns = col
    return df




In [3]:
#读取测试集和训练集

data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

#将响应变量单独列出
train_y = data_train['playtime_forever']
del data_train['playtime_forever']

myid = 'id'

#所有feature
#features = data_train.columns.tolist()

#所有干扰项feature
noisy_features = [myid]

In [4]:
#增加游戏上市和购买的年，月作为feature
#运行完毕后data_train 和 data_test 将修改

data_train['purchase_year'] = data_train['purchase_date'].apply(lambda x: int('20'+split(' |,|-',x)[2]) if isinstance(x,str) else 2017)
data_train['purchase_month'] = data_train['purchase_date'].apply(lambda x: split(' |,|-',x)[1] if isinstance(x,str) else 'Jan')
data_train['purchase_day'] = data_train['purchase_date'].apply(lambda x: int(split(' |,|-',x)[0]) if isinstance(x,str) else 1)

data_test['purchase_year'] = data_test['purchase_date'].apply(lambda x: int(split(' |,|-',x)[3]) if isinstance(x,str) else 2017)
data_test['purchase_month'] = data_test['purchase_date'].apply(lambda x: split(' |,|-',x)[0] if isinstance(x,str) else 'Jan')
data_test['purchase_day'] = data_test['purchase_date'].apply(lambda x: int(split(' |,|-',x)[1]) if isinstance(x,str) else 1)

data_train['release_year'] = data_train['release_date'].apply(lambda x: int(split(' |,|-',x)[3]) if isinstance(x,str) else 2017)
data_train['release_month'] = data_train['release_date'].apply(lambda x: split(' |,|-',x)[1] if isinstance(x,str) else 'Jan')
data_train['release_day'] = data_train['release_date'].apply(lambda x: int(split(' |,|-',x)[0]) if isinstance(x,str) else 1)

data_test['release_year'] = data_test['release_date'].apply(lambda x: int('20'+split('-',x)[2]) if isinstance(x,str) else 2017)
data_test['release_month'] = data_test['release_date'].apply(lambda x: split('-',x)[1] if isinstance(x,str) else 'Jan')
data_test['release_day'] = data_test['release_date'].apply(lambda x: int(split('-',x)[0]) if isinstance(x,str) else 1)

data_train = data_train.replace('Jan',int(1))
data_train = data_train.replace('Feb',int(2))
data_train = data_train.replace('Mar',int(3))
data_train = data_train.replace('Apr',int(4))
data_train = data_train.replace('May',int(5))
data_train = data_train.replace('Jun',int(6))
data_train = data_train.replace('Jul',int(7))
data_train = data_train.replace('Aug',int(8))
data_train = data_train.replace('Sep',int(9))
data_train = data_train.replace('Oct',int(10))
data_train = data_train.replace('Nov',int(11))
data_train = data_train.replace('Dec',int(12))

data_test = data_test.replace('Jan',int(1))
data_test = data_test.replace('Feb',int(2))
data_test = data_test.replace('Mar',int(3))
data_test = data_test.replace('Apr',int(4))
data_test = data_test.replace('May',int(5))
data_test = data_test.replace('Jun',int(6))
data_test = data_test.replace('Jul',int(7))
data_test = data_test.replace('Aug',int(8))
data_test = data_test.replace('Sep',int(9))
data_test = data_test.replace('Oct',int(10))
data_test = data_test.replace('Nov',int(11))
data_test = data_test.replace('Dec',int(12))

data_train['gap_day'] = abs((data_train['purchase_year'] - data_train['release_year']) * 365 + (data_train['purchase_month'] - data_train['release_month'])/12 + data_train['purchase_day'] - data_train['release_day'])
data_test['gap_day'] = abs((data_test['purchase_year'] - data_test['release_year']) * 365 + (data_test['purchase_month'] - data_test['release_month'])/12 + data_test['purchase_day'] - data_test['release_day'])
data_train['gap_year'] = abs(data_train['purchase_year'] - data_train['release_year'])
data_test['gap_year'] = abs(data_test['purchase_year'] - data_test['release_year'])
#noisy_features.extend(['purchase_date','release_date','purchase_day','release_day','purchase_month','release_month','purchase_year','release_year'])
noisy_features.extend(['purchase_date','release_date','purchase_day','release_day','purchase_month','release_month'])


In [5]:
#增加tags，categories和genres的one-hot分类
#运行完毕后data_train 和 data_test 将修改

data_train_tags = data_train['tags'].str.get_dummies(",") 
data_test_tags = data_test['tags'].str.get_dummies(",") 
data_train_categories = data_train['categories'].str.get_dummies(",") 
data_test_categories = data_test['categories'].str.get_dummies(",") 
data_train_genres = data_train['genres'].str.get_dummies(",") 
data_test_genres = data_test['genres'].str.get_dummies(",") 

data_train_tags, data_test_tags = intersect(data_train_tags, data_test_tags)
data_train_categories, data_test_categories = intersect(data_train_categories, data_test_categories)
data_train_genres, data_test_genres = intersect(data_train_genres, data_test_genres)

data_train_tags = col_addname(data_train_tags,'tags')
data_test_tags = col_addname(data_test_tags,'tags')
data_train_genres = col_addname(data_train_genres,'genres')
data_test_genres = col_addname(data_test_genres,'genres')
data_train_categories = col_addname(data_train_categories,'categories')
data_test_categories = col_addname(data_test_categories,'categories')

data_train_dummy = pd.concat([data_train_tags,data_train_categories,data_train_genres])
data_test_dummy = pd.concat([data_test_tags,data_test_categories,data_test_genres])
dummy_features = data_train_dummy.columns.tolist()


data_train = pd.concat([data_train, data_train_tags, data_train_categories, data_train_genres],axis=1)
data_test = pd.concat([data_test, data_test_tags, data_test_categories, data_test_genres],axis=1)

noisy_features.extend(['tags','categories','genres'])

In [6]:
#增删好评率相关的feature
#运行完毕后data_train 和 data_test 将修改

data_train['total_positive_reviews'] = data_train['total_positive_reviews'].fillna(method='ffill')
data_train['total_negative_reviews'] = data_train['total_negative_reviews'].fillna(method='ffill')
data_train['heat'] = data_train['total_positive_reviews'] + data_train['total_negative_reviews']
#data_train['total_reviews'] = data_train['total_positive_reviews'] + data_train['total_negative_reviews']
data_train['positive_rate'] = data_train['total_positive_reviews'] / (data_train['total_negative_reviews'] + 1)

data_test['total_positive_reviews'] = data_test['total_positive_reviews'].fillna(method='ffill')
data_test['total_negative_reviews'] = data_test['total_negative_reviews'].fillna(method='ffill')
#data_test['total_reviews'] = data_test['total_positive_reviews'] + data_test['total_negative_reviews']
data_test['positive_rate'] = data_test['total_positive_reviews'] / (data_test['total_negative_reviews'] + 1)
data_test['heat'] = data_test['total_positive_reviews'] + data_test['total_negative_reviews']

#noisy_features.extend(['total_negative_reviews','total_positive_reviews'])

In [7]:
#对 is_free feature 改造
#运行完毕后data_train 和 data_test 将修改

data_train['is_free'] = data_train['is_free'].astype(int)
data_test['is_free'] = data_test['is_free'].astype(int)

In [8]:
#对 price 改造
#运行完毕后data_train 和 data_test 将修改

data_train['price'] = data_train['price'].apply(lambda x: 40000 if x > 40000 else x)
data_test['price'] = data_test['price'].apply(lambda x: 40000 if x > 40000 else x)

In [9]:
#先对非dummy的feature进行分析

features = data_train.columns.tolist()
non_dummy_features = [f for f in features if f not in noisy_features and f not in dummy_features]
data_train_non_dummy = data_train[non_dummy_features]
data_test_non_dummy = data_test[non_dummy_features]

In [10]:
#再对dummy的feature进行分析

#features = data_train.columns.tolist()
data_train_dummy = data_train[dummy_features]
data_test_dummy = data_test[dummy_features]

print data_train_dummy.shape
print data_test_dummy.shape

(357, 266)
(90, 266)


In [11]:
data_train_dummy['playtime_forever'] = train_y#np.log(1+ train_y)
#data_train_non_dummy['playtime_forever'] = np.log(1+ train_y)

/home/mok/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
print data_train_dummy.columns

Index([u'categoriesCaptions available', u'categoriesCo-op',
       u'categoriesCommentary available',
       u'categoriesCross-Platform Multiplayer',
       u'categoriesFull controller support', u'categoriesIn-App Purchases',
       u'categoriesIncludes Source SDK', u'categoriesIncludes level editor',
       u'categoriesLocal Co-op', u'categoriesLocal Multi-Player',
       ...
       u'tagsVisual Novel', u'tagsWalking Simulator', u'tagsWar',
       u'tagsWarhammer 40K', u'tagsWestern', u'tagsWorld War I',
       u'tagsWorld War II', u'tagsZombies', u'tagseSports',
       u'playtime_forever'],
      dtype='object', length=267)


In [13]:
data_train_dummy['playtime_forever'].groupby(data_train_dummy['tagsAtmospheric']).mean()

tagsAtmospheric
0    2.524314
1    3.660071
Name: playtime_forever, dtype: float64

In [14]:
train_y = train_y.values

In [15]:
#筛选显著的dummy features
def filter_useful_dummy_feature(data_train_dummy,thre):
    dummy_useful = [""]

    for w in data_train_dummy.columns:
        if w == 'playtime_forever':
            continue
        tmp1 = data_train_dummy['playtime_forever'].groupby(data_train_dummy[w]).mean()[0]
        tmp2 = data_train_dummy['playtime_forever'].groupby(data_train_dummy[w]).mean()[1]
        Max = np.max([tmp1, tmp2])+1e-3
        Min = np.min([tmp1, tmp2])+1e-3 
        c = data_train[w].sum()
        #print w + ':' + str(Max/Min)
        if Max/Min > thre:
            dummy_useful.append(w)
    del dummy_useful[0]
    return dummy_useful

In [45]:
#筛选同分布的dummy features

def filter_samedis_dummy_features(data_train_dummy, data_test_dummy,thre):
    from sklearn import linear_model
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    dummy_samedis = [""]
    for w in data_train_dummy.columns:

        if w == 'playtime_forever':
            continue
        l1 = data_train_dummy.shape[0]
        l2 = data_test_dummy.shape[0]
        train = data_train_dummy[w].values
        test = data_test_dummy[w].values
        train_one = np.ones((l1,1))
        test_zero = np.zeros((l2,1))
        train = train.reshape((len(train),1))
        test = test.reshape((len(test),1))
        X = np.concatenate((train,test),axis=0)
        y = np.concatenate((train_one, test_zero),axis=0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  #2较好
        clf = linear_model.SGDClassifier(random_state=1)
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        #print w +':'+ str(accuracy_score(y_pred, y_test))
        if accuracy_score(y_pred, y_test)<=thre:
            dummy_samedis.append(w)
    del dummy_samedis[0]
    return dummy_samedis

In [75]:
dummy_useful = filter_useful_dummy_feature(data_train_dummy,1.5)
dummy_samedis = filter_samedis_dummy_features(data_train_dummy, data_test_dummy,0.78)
dummy_features= [i for i in dummy_samedis if i in dummy_useful]

In [76]:
dummy_features = [i for i in dummy_useful if i in dummy_samedis]

In [77]:
features_final = dummy_features + non_dummy_features
train_x = data_train[features_final]
test_x = data_test[features_final]
print len(dummy_useful)
print len(dummy_samedis)
print len(features_final)

213
122
111


In [78]:
weight = np.log(train_y + 5) 

In [79]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.01)

regr = RandomForestRegressor(max_depth=100,n_estimators=1300,min_samples_split=3,random_state=3)
#regr.fit(X_train, y_train,sample_weight=weight)
y = train_y ** 2
regr.fit(train_x, y,sample_weight=weight)
#y_val = np.exp(y_val) - 1
#print accuracy_score(y_test, y_val)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           n_estimators=1300, n_jobs=1, oob_score=False, random_state=3,
           verbose=0, warm_start=False)

In [80]:
test_y = regr.predict(test_x)
test_y = test_y ** 0.5
for i in range(len(test_y)):
    if test_y[i] < 13:
        test_y[i] = test_y[i] ** 0.5


In [81]:
df = pd.DataFrame({'id':range(len(test_y)),'playtime_forever':test_y})

In [82]:
#df.to_csv('submission.csv',index=False)